# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

[2025-05-06 17:33:47] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=30982, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=675496266, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_

[2025-05-06 17:33:57] Attention backend not set. Use fa3 backend by default.
[2025-05-06 17:33:57] Init torch distributed begin.
[2025-05-06 17:33:57] Init torch distributed ends. mem usage=0.00 GB
[2025-05-06 17:33:57] Load weight begin. avail mem=53.74 GB


[2025-05-06 17:33:57] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-06 17:33:58] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.24s/it]


...

[2025-05-06 17:34:01] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-06 17:34:01] INFO:     Started server process [3962001]
[2025-05-06 17:34:01] INFO:     Waiting for application startup.
[2025-05-06 17:34:01] INFO:     Application startup complete.
[2025-05-06 17:34:01] INFO:     Uvicorn running on http://0.0.0.0:30982 (Press CTRL+C to quit)


[2025-05-06 17:34:02] INFO:     127.0.0.1:37508 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-06 17:34:02] INFO:     127.0.0.1:37522 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-06 17:34:02] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-06 17:34:05] INFO:     127.0.0.1:37534 - "POST /generate HTTP/1.1" 200 OK
[2025-05-06 17:34:05] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Please generate the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-06 17:34:07] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-06 17:34:08] Decode batch. #running-req: 1, #token: 52, token usage: 0.00, gen throughput (token/s): 5.35, #queue-req: 0


[2025-05-06 17:34:09] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, gen throughput (token/s): 104.82, #queue-req: 0


[2025-05-06 17:34:09] Decode batch. #running-req: 1, #token: 132, token usage: 0.01, gen throughput (token/s): 106.30, #queue-req: 0


[2025-05-06 17:34:09] Decode batch. #running-req: 1, #token: 172, token usage: 0.01, gen throughput (token/s): 106.34, #queue-req: 0


...

[2025-05-06 17:34:26] Decode batch. #running-req: 1, #token: 1892, token usage: 0.09, gen throughput (token/s): 95.88, #queue-req: 0


[2025-05-06 17:34:26] Decode batch. #running-req: 1, #token: 1932, token usage: 0.09, gen throughput (token/s): 103.36, #queue-req: 0


[2025-05-06 17:34:27] Decode batch. #running-req: 1, #token: 1972, token usage: 0.10, gen throughput (token/s): 101.02, #queue-req: 0


[2025-05-06 17:34:27] Decode batch. #running-req: 1, #token: 2012, token usage: 0.10, gen throughput (token/s): 101.65, #queue-req: 0


[2025-05-06 17:34:28] Decode batch. #running-req: 1, #token: 2052, token usage: 0.10, gen throughput (token/s): 104.72, #queue-req: 0
[2025-05-06 17:34:28] INFO:     127.0.0.1:38926 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Give me the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-06 17:34:28] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-06 17:34:28] Decode batch. #running-req: 1, #token: 44, token usage: 0.00, gen throughput (token/s): 72.60, #queue-req: 0


[2025-05-06 17:34:28] Decode batch. #running-req: 1, #token: 84, token usage: 0.00, gen throughput (token/s): 103.85, #queue-req: 0


[2025-05-06 17:34:29] Decode batch. #running-req: 1, #token: 124, token usage: 0.01, gen throughput (token/s): 66.06, #queue-req: 0


[2025-05-06 17:34:30] Decode batch. #running-req: 1, #token: 164, token usage: 0.01, gen throughput (token/s): 65.95, #queue-req: 0


...

[2025-05-06 17:34:50] Decode batch. #running-req: 1, #token: 1924, token usage: 0.09, gen throughput (token/s): 103.03, #queue-req: 0


[2025-05-06 17:34:51] Decode batch. #running-req: 1, #token: 1964, token usage: 0.10, gen throughput (token/s): 103.57, #queue-req: 0


[2025-05-06 17:34:51] Decode batch. #running-req: 1, #token: 2004, token usage: 0.10, gen throughput (token/s): 102.78, #queue-req: 0


[2025-05-06 17:34:52] Decode batch. #running-req: 1, #token: 2044, token usage: 0.10, gen throughput (token/s): 90.57, #queue-req: 0


[2025-05-06 17:34:52] INFO:     127.0.0.1:38926 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "user",
            "content": "Give me the information of the capital of France.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-06 17:34:52] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-06 17:34:52] Decode batch. #running-req: 1, #token: 39, token usage: 0.00, gen throughput (token/s): 87.45, #queue-req: 0


[2025-05-06 17:34:52] Decode batch. #running-req: 1, #token: 79, token usage: 0.00, gen throughput (token/s): 105.10, #queue-req: 0


[2025-05-06 17:34:53] Decode batch. #running-req: 1, #token: 119, token usage: 0.01, gen throughput (token/s): 105.55, #queue-req: 0
[2025-05-06 17:34:53] INFO:     127.0.0.1:38926 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-06 17:34:53] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-06 17:34:53] Decode batch. #running-req: 1, #token: 33, token usage: 0.00, gen throughput (token/s): 96.29, #queue-req: 0


[2025-05-06 17:34:54] Decode batch. #running-req: 1, #token: 73, token usage: 0.00, gen throughput (token/s): 105.51, #queue-req: 0


[2025-05-06 17:34:54] Decode batch. #running-req: 1, #token: 113, token usage: 0.01, gen throughput (token/s): 105.55, #queue-req: 0


[2025-05-06 17:34:54] Decode batch. #running-req: 1, #token: 153, token usage: 0.01, gen throughput (token/s): 104.43, #queue-req: 0


[2025-05-06 17:34:55] Decode batch. #running-req: 1, #token: 193, token usage: 0.01, gen throughput (token/s): 97.88, #queue-req: 0


[2025-05-06 17:34:55] Decode batch. #running-req: 1, #token: 233, token usage: 0.01, gen throughput (token/s): 99.68, #queue-req: 0


[2025-05-06 17:34:56] Decode batch. #running-req: 1, #token: 273, token usage: 0.01, gen throughput (token/s): 100.16, #queue-req: 0


[2025-05-06 17:34:56] Decode batch. #running-req: 1, #token: 313, token usage: 0.02, gen throughput (token/s): 105.86, #queue-req: 0


[2025-05-06 17:34:56] Decode batch. #running-req: 1, #token: 353, token usage: 0.02, gen throughput (token/s): 101.27, #queue-req: 0
[2025-05-06 17:34:56] INFO:     127.0.0.1:38926 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "user",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-06 17:34:57] Prefill batch. #new-seq: 1, #new-token: 471, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-06 17:34:57] Decode batch. #running-req: 1, #token: 495, token usage: 0.02, gen throughput (token/s): 37.98, #queue-req: 0


[2025-05-06 17:34:58] Decode batch. #running-req: 1, #token: 535, token usage: 0.03, gen throughput (token/s): 103.95, #queue-req: 0


[2025-05-06 17:34:58] Decode batch. #running-req: 1, #token: 575, token usage: 0.03, gen throughput (token/s): 102.42, #queue-req: 0


[2025-05-06 17:34:59] Decode batch. #running-req: 1, #token: 615, token usage: 0.03, gen throughput (token/s): 100.71, #queue-req: 0


[2025-05-06 17:34:59] Decode batch. #running-req: 1, #token: 655, token usage: 0.03, gen throughput (token/s): 101.94, #queue-req: 0


[2025-05-06 17:34:59] Decode batch. #running-req: 1, #token: 695, token usage: 0.03, gen throughput (token/s): 101.23, #queue-req: 0


[2025-05-06 17:35:00] Decode batch. #running-req: 1, #token: 735, token usage: 0.04, gen throughput (token/s): 101.36, #queue-req: 0


[2025-05-06 17:35:00] Decode batch. #running-req: 1, #token: 775, token usage: 0.04, gen throughput (token/s): 101.07, #queue-req: 0


[2025-05-06 17:35:00] Decode batch. #running-req: 1, #token: 815, token usage: 0.04, gen throughput (token/s): 100.98, #queue-req: 0
[2025-05-06 17:35:01] INFO:     127.0.0.1:38926 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "user",
        "content": "Here is the information of the capital of France in the JSON format.\n",
    }
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-05-06 17:35:01] Prefill batch. #new-seq: 1, #new-token: 19, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-06 17:35:02] Decode batch. #running-req: 1, #token: 56, token usage: 0.00, gen throughput (token/s): 37.99, #queue-req: 0


[2025-05-06 17:35:02] Decode batch. #running-req: 1, #token: 96, token usage: 0.00, gen throughput (token/s): 103.28, #queue-req: 0


[2025-05-06 17:35:02] Decode batch. #running-req: 1, #token: 136, token usage: 0.01, gen throughput (token/s): 105.06, #queue-req: 0


[2025-05-06 17:35:03] Decode batch. #running-req: 1, #token: 176, token usage: 0.01, gen throughput (token/s): 104.31, #queue-req: 0


...

[2025-05-06 17:35:20] Decode batch. #running-req: 1, #token: 1896, token usage: 0.09, gen throughput (token/s): 89.80, #queue-req: 0


[2025-05-06 17:35:21] Decode batch. #running-req: 1, #token: 1936, token usage: 0.09, gen throughput (token/s): 102.92, #queue-req: 0


[2025-05-06 17:35:21] Decode batch. #running-req: 1, #token: 1976, token usage: 0.10, gen throughput (token/s): 102.55, #queue-req: 0


[2025-05-06 17:35:21] Decode batch. #running-req: 1, #token: 2016, token usage: 0.10, gen throughput (token/s): 102.72, #queue-req: 0


[2025-05-06 17:35:22] Decode batch. #running-req: 1, #token: 2056, token usage: 0.10, gen throughput (token/s): 102.44, #queue-req: 0
[2025-05-06 17:35:22] INFO:     127.0.0.1:40656 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so I need to provide the information about the capital of France in JSON format. Hmm, I\'m not entirely sure about all the details, but I\'ll try to think it through.\n\nFirst, I know that the capital of France is Paris. That\'s pretty much a given, right? But I should double-check that. Maybe I can recall any other capitals I know. London is the capital of the UK, Rome is Italy, and maybe Tokyo is Japan\'s. Yeah, Paris seems correct for France.\n\nNow, moving on to the population. I think Paris is a very large city, but I\'m not sure of the exact number. I remember it\'s over 3 million, but I\'m not certain. Maybe around 3.5 million? I should probably look that up, but since I can\'t right now, I\'ll go with 3,500,000 as an estimate.\n\nNext, the area. Par

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(text, tokenize=False, add_generation_prompt=True)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-05-06 17:35:22] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-06 17:35:22] Decode batch. #running-req: 1, #token: 33, token usage: 0.00, gen throughput (token/s): 98.26, #queue-req: 0


[2025-05-06 17:35:23] Decode batch. #running-req: 1, #token: 73, token usage: 0.00, gen throughput (token/s): 105.84, #queue-req: 0


[2025-05-06 17:35:23] Decode batch. #running-req: 1, #token: 113, token usage: 0.01, gen throughput (token/s): 106.04, #queue-req: 0


[2025-05-06 17:35:23] Decode batch. #running-req: 1, #token: 153, token usage: 0.01, gen throughput (token/s): 105.96, #queue-req: 0


...

[2025-05-06 17:35:28] Decode batch. #running-req: 1, #token: 633, token usage: 0.03, gen throughput (token/s): 103.51, #queue-req: 0


[2025-05-06 17:35:28] Decode batch. #running-req: 1, #token: 673, token usage: 0.03, gen throughput (token/s): 102.75, #queue-req: 0


[2025-05-06 17:35:29] Decode batch. #running-req: 1, #token: 713, token usage: 0.03, gen throughput (token/s): 104.08, #queue-req: 0


[2025-05-06 17:35:29] Decode batch. #running-req: 1, #token: 753, token usage: 0.04, gen throughput (token/s): 98.77, #queue-req: 0


[2025-05-06 17:35:29] Decode batch. #running-req: 1, #token: 793, token usage: 0.04, gen throughput (token/s): 101.71, #queue-req: 0
[2025-05-06 17:35:30] INFO:     127.0.0.1:42412 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-05-06 17:35:30] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-06 17:35:30] Prefill batch. #new-seq: 3, #new-token: 3, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-06 17:35:30] Decode batch. #running-req: 3, #token: 122, token usage: 0.01, gen throughput (token/s): 175.57, #queue-req: 0


[2025-05-06 17:35:31] Decode batch. #running-req: 3, #token: 242, token usage: 0.01, gen throughput (token/s): 299.23, #queue-req: 0


[2025-05-06 17:35:31] Decode batch. #running-req: 3, #token: 362, token usage: 0.02, gen throughput (token/s): 299.23, #queue-req: 0


[2025-05-06 17:35:31] Decode batch. #running-req: 3, #token: 482, token usage: 0.02, gen throughput (token/s): 287.00, #queue-req: 0


[2025-05-06 17:35:32] Decode batch. #running-req: 3, #token: 602, token usage: 0.03, gen throughput (token/s): 295.35, #queue-req: 0
[2025-05-06 17:35:32] INFO:     127.0.0.1:49548 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris.\n\nThat's all the information I have.\n\nOkay, so I need to figure out the capital of France. I know that Paris is the capital, but I'm not entirely sure. Let me think about why I think that. I've heard it mentioned a lot, especially in movies and TV shows. People often go there for business or tourism. Also, I remember learning in school that Paris is a major city in France, known for landmarks like the Eiffel Tower and the Louvre Museum. Those places are famous worldwide, which makes me think that Paris is indeed the capital. Maybe I can cross-check this with some other sources or my notes. Wait, I don't have any other information right now, but based on what I know, Paris is the capital of France. I don't recall any other majo

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-05-06 17:35:32] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-06 17:35:32] Decode batch. #running-req: 1, #token: 42, token usage: 0.00, gen throughput (token/s): 121.09, #queue-req: 0


[2025-05-06 17:35:33] Decode batch. #running-req: 1, #token: 82, token usage: 0.00, gen throughput (token/s): 107.00, #queue-req: 0


[2025-05-06 17:35:33] Decode batch. #running-req: 1, #token: 122, token usage: 0.01, gen throughput (token/s): 107.42, #queue-req: 0


[2025-05-06 17:35:33] Decode batch. #running-req: 1, #token: 162, token usage: 0.01, gen throughput (token/s): 107.35, #queue-req: 0


...

[2025-05-06 17:35:50] Decode batch. #running-req: 1, #token: 1882, token usage: 0.09, gen throughput (token/s): 104.21, #queue-req: 0


[2025-05-06 17:35:50] Decode batch. #running-req: 1, #token: 1922, token usage: 0.09, gen throughput (token/s): 104.31, #queue-req: 0


[2025-05-06 17:35:50] Decode batch. #running-req: 1, #token: 1962, token usage: 0.10, gen throughput (token/s): 104.33, #queue-req: 0


[2025-05-06 17:35:51] Decode batch. #running-req: 1, #token: 2002, token usage: 0.10, gen throughput (token/s): 104.34, #queue-req: 0


[2025-05-06 17:35:51] Decode batch. #running-req: 1, #token: 2042, token usage: 0.10, gen throughput (token/s): 104.30, #queue-req: 0
[2025-05-06 17:35:51] INFO:     127.0.0.1:49556 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\(

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-05-06 17:35:51] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 19, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-06 17:35:52] Decode batch. #running-req: 1, #token: 48, token usage: 0.00, gen throughput (token/s): 99.11, #queue-req: 0


[2025-05-06 17:35:52] Decode batch. #running-req: 1, #token: 88, token usage: 0.00, gen throughput (token/s): 105.92, #queue-req: 0


[2025-05-06 17:35:52] Decode batch. #running-req: 1, #token: 128, token usage: 0.01, gen throughput (token/s): 105.65, #queue-req: 0


[2025-05-06 17:35:53] Decode batch. #running-req: 1, #token: 168, token usage: 0.01, gen throughput (token/s): 105.77, #queue-req: 0


[2025-05-06 17:35:53] Decode batch. #running-req: 1, #token: 208, token usage: 0.01, gen throughput (token/s): 104.61, #queue-req: 0


[2025-05-06 17:35:53] INFO:     127.0.0.1:38472 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.41s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.29s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.30s/it]



### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure, here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Tiananmen Square",
    "Forbidden City",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 
The capital of France is Paris. Let me know if you need more information!
Prompt: Give me the information of the capital of Germany.
Generated text: 
The capital of Germany is Berlin. If you have any other questions, feel free to ask!
</think>Berlin is the capital of Germany
Prompt: Give me the information of the capital of Italy.
Generated text: 
The capital of Italy is Rome. Here, I will provide the information in a clear and organized way.

First, the capital of Italy is Rome. It is located in central Italy, on the Tevere River.

Rome is a very old city. It has a rich history dating back over 2,000 years.

The city is known for its unique blend of historical, cultural, and artistic traditions.

Rome is at sea level, so it is subject to the effects of the sea on urban planning.

The city is divided into several historical districts, each with its own distinct character.

Rome has been the


### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its location, economic role, cultural significance, and the impact of COVID-19 on London.
London is one of the most significant cities in the world, located at the con confluence of the River Thames and the River Medway. It serves as the capital of the United Kingdom and has been a major political, economic, and cultural center for centuries. The city is home to a diverse population and is known for its historic landmarks, modern architecture, and vibrant nightlife. During the COVID-19 pandemic, London faced significant challenges with its economy, transportation, and social life. The city went into a strict lockdown to curb the spread
Prompt: Please provide information about Paris as a major global city:
Generated text:  its location, history, current status, and the role of Paris in the global economy.

 Paris is one of the most important cities in the world, known for its rich history and its ro

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜User｜>Here is the information of the capital of France in the JSON format.
<｜Assistant｜><think>

Generated text: Alright, so I need to figure out the capital of France and present it in JSON format. Hmm, I'm not entirely sure, but I think Paris is the capital. Let me just double-check that to be sure. Yeah, I'm pretty confident Paris is the official capital city of France. 

Now, the user has provided an example JSON response, so I should follow that structure. They listed the capital as "Paris" and included some related information like population, area, and location. I should make sure to include similar details to make it comprehensive.

I wonder, what other details should I include? Maybe the administrative region, like Île-de-France, and its significance. Also, perhaps mention that it's the most populous city in France. That would make the information more accurate and informative.

Let me think about the structure again. The example had a "name" key

In [19]:
llm.shutdown()